In [ ]:
import pandas as pd 
import numpy as np
import glob
import json
import xlrd
import datetime
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
station_list = [
    '문산','파주','월롱','금촌','금릉','운정',
    '야당','탄현','일산','풍산','백마','곡산',
    '대곡','능곡','행신','강매','화전','수색',
    '디지털미디어시티','가좌','홍대입구','서강대','공덕',
    '효창공원앞','용산','이촌','서빙고','한남',
    '옥수','응봉','왕십리','청량리','회기',
    '중랑','상봉','망우','양원','구리',
    '도농','양정','덕소','도심','팔당',
    '운길산','양수','신원','국수','아신',
    '오빈','양평','원덕','용문','지평'
]

convert_dict = {
    '디엠시':'디지털미디어시티',
    '홍대입':'홍대입구',
    '효창공':'효창공원앞',
    '1양원':'양원',
    '1양정':'양정'
}

train_list_dict = {
    6: {'5010','5014','5016','5012','5018','5020','5702'},
    7: {'5022','5024','5026','5704','5302','5028','5706','5030'},
    8: {'5032','5034','5232','5038','5040','5036'},
    9: {'5044','5046','5042','5048','5050'}
}

In [ ]:
# 샘플 데이터를 사용합니다. 직접 크롤링 한 경우 path 를 변경합니다.
path = "./sampledata/*"
file_list = glob.glob(path)
file_list.sort()

In [ ]:
# 데이터를 로드합니다.
# 같은 시간 축에 표시하게 하기 위해 같은 날짜로 고정하니 필요하면 그 부분을 수정해야 합니다.

train_dict = dict()

for each_file in file_list:
    # 주말 데이터를 제거합니다. 필요시 코드를 제거합니다.
    if each_file.split('-')[1:3] in [['09','12'],['09','13'],['19','20']]:
        continue
    with open(each_file) as json_file:
        json_data = json.load(json_file)
        for each in json_data:
            if each['trainNo'] in ['5172','5168','5160','5156']: # Crawling Error
                continue
            if each['trainNo'][0] == '2': # 경의선 데이터 제외
                continue
            if each['updnLine'] == "0": # 상행선 데이터 제외
                continue

            if each['trainNo'] not in train_dict:
                train_dict[each['trainNo']] = dict()

            train_date, train_time = each['recptnDt'].split(' ', 1)

            if train_date not in train_dict[each['trainNo']]:
                train_dict[each['trainNo']][train_date] = dict()

            train_dict[each['trainNo']][train_date][each['statnNm']] = datetime.datetime.strptime('2020-09-20 ' + train_time, '%Y-%m-%d %H:%M:%S')

In [ ]:
# Load Time table (계획된 시간표 로드)
xlsx = pd.read_excel('./timeline.xlsx', sheet_name=3)
xlsx.columns = xlsx.iloc[2]
xlsx = xlsx.iloc[3:107,1:]

In [ ]:
# Input: 열차번호
# Output: 열차의 시간표 데이터 dataframe
def get_timetable(apiTrainNo):
    dict_traintime = dict()
    trainNo = 'K' + str(int(apiTrainNo)-1)
    xlsx_traintime = xlsx.loc[:,['열차번호',trainNo]]

    for idx in xlsx_traintime.index:
        if idx % 2 == 0:
            continue

        time_arrive = xlsx_traintime._get_value(idx,trainNo).strip() if type(xlsx_traintime._get_value(idx,trainNo)) == type('a') else xlsx_traintime._get_value(idx,trainNo)
        time_depart = xlsx_traintime._get_value(idx+1,trainNo).strip() if type(xlsx_traintime._get_value(idx+1,trainNo)) == type('a') else xlsx_traintime._get_value(idx+1,trainNo)

        station_value = xlsx_traintime._get_value(idx,'열차번호')
        station_name = convert_dict[station_value] if station_value in convert_dict else station_value

        if (not time_arrive or pd.isna(time_arrive)) and (not time_depart or pd.isna(time_depart)):
            dict_traintime[station_name] = None
            continue

        train_time = time_arrive if type(time_depart) != datetime.time else time_depart
        dict_traintime[station_name] = datetime.datetime.combine(datetime.date(2020, 9, 20), train_time)

    output = pd.DataFrame.from_dict(dict_traintime, orient='index', columns=['timetable'])
    return output

In [ ]:
# Input: 열차번호
# Output: 실제 데이터와 시간표 데이터가 합쳐진 dataframe
def get_full_table(apiTrainNo):
    real_time = pd.DataFrame(train_dict[apiTrainNo])
    plan_time = get_timetable(apiTrainNo)
    merge_time = pd.concat([plan_time, real_time], axis=1)
    return merge_time

In [ ]:
# Input: 열차번호
# Output: 시간표 데이터와 차이
def get_diff(apiTrainNo):
    merge_time = get_full_table(apiTrainNo)
    
    for idx in merge_time.columns:
        if idx == 'timetable':
            continue
        merge_time[idx] = (merge_time['timetable'] - merge_time[idx]).dt.total_seconds()
    merge_time = merge_time.drop(labels='timetable', axis=1)
    return merge_time

In [ ]:
# Input: 열차번호
# 열차의 도착시간을 선 그래프로 시각화
def show_single(apiTrainNo):
    merge_time = get_full_table(apiTrainNo)
    
    fig = px.line(merge_time, x=merge_time.columns, y=merge_time.index)
    fig['layout']['yaxis']['autorange'] = "reversed"
    fig.show()

In [ ]:
# Input: 열차번호
# 열차의 계획표와 실제 도착시간의 차이를 막대 그래프로 시각화
def show_diff(apiTrainNo):
    merge_time = get_diff(apiTrainNo)
   
    fig = go.Figure()
    for idx in merge_time.columns:
        fig.add_trace(go.Bar(x=merge_time.index, y=merge_time[idx],
                name=idx))
    fig.show()

In [ ]:
# Input: 열차번호
# 열차의 계획표와 실제 도착시간의 차이를 Box Plot 으로 시각화
def show_box(apiTrainNo):
    df_diff = get_diff(apiTrainNo)
    fig = px.box(df_diff, x=df_diff.index,  y=df_diff.columns)
    fig.show()

In [ ]:
# Input: 열차번호
# Output: 역간 이동 시간의 계획과의 차이 Dataframe
def get_diff_time(apiTrainNo):
    df = get_diff_station(apiTrainNo)
    df = df.fillna(0)

    for idx in df.columns:
        if idx == 'timetable':
            continue
        df.loc[df[idx] == 0, idx] = df.timetable
        df[idx] = (df['timetable'] - df[idx])

    df = df.drop(labels='timetable', axis=1)
    return df

In [ ]:
# Input: 열차번호
# Output: 역간 이동 시간 Dataframe
def get_diff_station(apiTrainNo):
    merge_time = get_full_table(apiTrainNo)
    
    merge_time = merge_time.ffill()
    merge_time = merge_time.diff()

    for column in merge_time.columns:
        merge_time[column] = merge_time[column].dt.total_seconds()
    merge_time = merge_time.clip(lower=0)
    
    return merge_time

In [ ]:
# Input: 열차번호
# 역간 이동 시간의 기존 시간표와 차이 시각화
def show_diff_time(apiTrainNo):
    df = get_diff_time(apiTrainNo)

    fig = go.Figure()
    for idx in df.columns:
        fig.add_trace(go.Bar(x=df.index, y=df[idx], base=0, name=idx))
    fig.show()

In [ ]:
show_single('5024')

In [ ]:
show_diff('5302')

In [ ]:
show_box('5034')